In [ ]:
# ==================================================================================================
# Yufei Li
# Princeton University
# yl5385@princeton.edu

# Feburay 2023

# Note:In this demo, the neural network is synthesized using the TensorFlow (verion: 2.11.0) framework. 
# Please install TensorFlow according to the official guidance, then import TensorFlow and other dependent modules.
# ==================================================================================================

# Setup environment
!pip install pandas numpy matplotlib
!pip install tensorflow

In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import copy
import csv
import math
import cmath
import time
import random as python_random

# reproduciable results
def reset_seeds():
   np.random.seed(7) 
   python_random.seed(7)
   tf.random.set_seed(7)  

inFilename = "Input_Yo20op1084_ana_bw20_50.csv"
outFilename = "Output_Yo20op1084_ana_bw20_50.csv"
input = pd.read_csv(inFilename,header=None)
output = pd.read_csv(outFilename,header=None)
inputRawTrain = []
outputRawTrain = []
inputRawTrain = np.array(input)
outputRawTrain = np.array(output)

inFilename = "Input_Yo20op40_ana_50.csv"
outFilename = "Output_Yo20op40_ana_50.csv"
input = pd.read_csv(inFilename,header=None)
output = pd.read_csv(outFilename,header=None)
inputTrain_1 = []
outputTrain_1 = []
inputTrain_1 = np.array(input)
outputTrain_1 = np.array(output)

num_inputs_train = len(inputTrain_1)
print("Total Number of Training Dataset is:",num_inputs_train)

# Shuffle the dataset
np.random.seed(1)
np.random.shuffle(inputTrain_1)
np.random.seed(1)
np.random.shuffle(outputTrain_1)

# Data separation
TRAIN_SPLIT_1 = int(0.7 * num_inputs_train)
TRAIN_SPLIT_2 = int(0.85 * num_inputs_train)
inputTrain, inputTest, inputs_test_2 = np.split(inputTrain_1, [TRAIN_SPLIT_1, TRAIN_SPLIT_2])
outputTrain, outputTest, outputs_test_2 = np.split(outputTrain_1, [TRAIN_SPLIT_1, TRAIN_SPLIT_2])

print("Dataset preparation complete!")

In [ ]:
CYCLE = 10
# Randomize the order of the inputs, so they can be evenly distributed for training, testing, and validation

num_inputRawTrain = len(inputRawTrain)
num_inputTrain = len(inputTrain)
num_inputTest = len(inputTest)
inputs_test = inputTest
outputs_test = outputTest

print("Total Number of Raw Training Dataset is:",num_inputRawTrain)
print("Total Number of Real Training Dataset is:",num_inputTrain)
print("Total Number of Test Dataset is:",num_inputTest)


inputs_train_raw = []
outputs_train_raw = []
inputs_train_real = []
outputs_train_real = []


for x in range(CYCLE):
  tempInput_raw = copy.deepcopy(inputRawTrain)
  tempOutput_raw = copy.deepcopy(outputRawTrain)
  tempInput_real = copy.deepcopy(inputTrain)
  tempOutput_real = copy.deepcopy(outputTrain)

  np.random.seed(x+1)
  np.random.shuffle(tempInput_raw)
  np.random.seed(x+1)
  np.random.shuffle(tempOutput_raw)
  inputs_train_raw.append(tempInput_raw)
  outputs_train_raw.append(tempOutput_raw)
    
  np.random.seed(x+1)
  np.random.shuffle(tempInput_real)
  np.random.seed(x+1)
  np.random.shuffle(tempOutput_real)
  inputs_train_real.append(tempInput_real)
  outputs_train_real.append(tempOutput_real)

print("Dataset randomization and separation complete!")

In [ ]:
# Set learning rate
initial_learning_rate = 0.020669824682365133
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=375*65,
    decay_rate=0.575401007701697,
    staircase=True)

# Normalization layer definition
Normlayer1=tf.keras.layers.Normalization()
Normlayer1.adapt(inputRawTrain)

# build and then train the model
def train_model(model,inputs_train,outputs_train,num_epoch):
    model.add(Normlayer1)
    model.add(tf.keras.layers.Dense(43, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(56, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(43, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(8))
    opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    model.compile(optimizer=opt, loss='mse', metrics=['mse'])

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
#       save_weights_only=True,
        verbose = 1,
        monitor='val_mse',
        mode='auto',
        save_best_only=True)

    history = model.fit(inputs_train, outputs_train, epochs=num_epoch, batch_size=16, validation_data=(inputs_test, outputs_test), callbacks=[model_checkpoint_callback])
    return history

# Use the model to predict test data
def predict_with_FNN(model,inputs_test,outputs_test):
    output_pred = model.predict(inputs_test)
    output_diff = output_pred - outputs_test
    AE = output_diff
    MAE = np.mean(abs(output_diff))
    MAE_1 =  np.mean(abs(output_pred[:,[0,2,4,6]] - outputs_test[:,[0,2,4,6]]))
    MAE_2 =  np.mean(abs(output_pred[:,[1,3,5,7]] - outputs_test[:,[1,3,5,7]]))
    MSE = np.square(np.subtract(output_pred,outputs_test)).mean()
    MSE_1 = np.square(np.subtract(output_pred[:,[0,2,4,6]],outputs_test[:,[0,2,4,6]])).mean()
    MSE_2 = np.square(np.subtract(output_pred[:,[1,3,5,7]],outputs_test[:,[1,3,5,7]])).mean()
    
    pct_95th_error = np.percentile(abs(output_diff),95)
    std_error = np.std(output_diff)
    print("Mean Absolute Error(MAE): %.9f" % MAE)
    print("MAE conductance: %.9f" % MAE_1)
    print("MAE susceptance: %.9f" % MAE_2)
    print("95th Error: %.9f" % pct_95th_error)
    print("Standard Deviation: %.9f" % std_error)
    print("MSE: %.9f" % MSE)
    print("MSE conductance: %.9f" % MSE_1)
    print("MSE susceptance: %.9f" % MSE_2)
    print("Absolute Error(first element, conductance):", np.around(AE[[0],[0,2,4,6]],9))

    return MAE,MAE_1,MAE_2,pct_95th_error,std_error,MSE,MSE_1,MSE_2,AE

print("Training and prediction function definition complete!")

In [ ]:
EPOCHS = 500

# initialization/getting reproducible results
reset_seeds()

# Direct Train
mae_direct = []
mae_direct_iteration = []
mae_1_direct = []
mae_1_direct_iteration = []
mae_2_direct = []
mae_2_direct_iteration = []
pct95th_direct = []
pct95th_direct_iteration = []
std_direct = []
std_direct_iteration = []
mse_direct = []
mse_direct_iteration = []
mse_1_direct = []
mse_1_direct_iteration = []
mse_2_direct = []
mse_2_direct_iteration = []
ae_direct = []
ae_direct_iteration = []

num_data_array = [5,10,30,50,100,150,200,300,400,550]

for x in range(len(num_data_array)):
    # Set learning rate
    initial_learning_rate = 0.020669824682365133
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=math.ceil(num_data_array[x]/16)*65,
        decay_rate=0.575401007701697,
        staircase=True)
    
    mae_direct_iteration.append(num_data_array[x])
    mae_1_direct_iteration.append(num_data_array[x])
    mae_2_direct_iteration.append(num_data_array[x])
    pct95th_direct_iteration.append(num_data_array[x])
    std_direct_iteration.append(num_data_array[x])
    mse_direct_iteration.append(num_data_array[x])
    mse_1_direct_iteration.append(num_data_array[x])
    mse_2_direct_iteration.append(num_data_array[x])
    ae_direct_iteration.append(num_data_array[x])
    for num_train_iteration in range(10):
        inputs_train_real_cut = inputs_train_real[num_train_iteration][0:0+num_data_array[x]]
        outputs_train_real_cut = outputs_train_real[num_train_iteration][0:0+num_data_array[x]]
#         inputs_train_real_cut = inputs_train_real[num_train_iteration]
#         outputs_train_real_cut = outputs_train_real[num_train_iteration]
        
        model_direct = tf.keras.Sequential()
        checkpoint_filepath = './checkpoint/direct/direct_weights.hdf5' #
        history = train_model(model_direct,inputs_train_real_cut,outputs_train_real_cut,EPOCHS)
        model_direct.load_weights(checkpoint_filepath) # load the best model
        predict_results = predict_with_FNN(model_direct,inputs_test,outputs_test)
        print("Number of Data Trained: %.1f" % num_data_array[x])
        print("Iteration Times: %.1f" % num_train_iteration)
        
        mae_direct_iteration.append(predict_results[0])
        mae_1_direct_iteration.append(predict_results[1])
        mae_2_direct_iteration.append(predict_results[2])
        pct95th_direct_iteration.append(predict_results[3])
        std_direct_iteration.append(predict_results[4])
        mse_direct_iteration.append(predict_results[5])
        mse_1_direct_iteration.append(predict_results[6])
        mse_2_direct_iteration.append(predict_results[7])
        for i in range(num_inputTest):
            ae_direct_iteration.extend(predict_results[8][i])
        print()
        
    mae_direct.append(mae_direct_iteration)
    mae_direct_iteration = []
    mae_1_direct.append(mae_1_direct_iteration)
    mae_1_direct_iteration = []
    mae_2_direct.append(mae_2_direct_iteration)
    mae_2_direct_iteration = []
    pct95th_direct.append(pct95th_direct_iteration)
    pct95th_direct_iteration = []
    std_direct.append(std_direct_iteration)
    std_direct_iteration = []
    mse_direct.append(mse_direct_iteration)
    mse_direct_iteration = []
    mse_1_direct.append(mse_1_direct_iteration)
    mse_1_direct_iteration = []
    mse_2_direct.append(mse_2_direct_iteration)
    mse_2_direct_iteration = []
    ae_direct.append(ae_direct_iteration)
    ae_direct_iteration = []

In [ ]:
# increase the size of the graphs. The default size is (6,4).
plt.rcParams["figure.figsize"] = (9,6)

# list all data in history
print(history.history.keys())

# summarize history for loss
plt.plot(history.history['loss'], '-r.', label='Training Loss')
plt.plot(history.history['val_loss'], '--b.', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.semilogy()
plt.grid()
plt.show()

print(plt.rcParams["figure.figsize"])
#print(history.history)

In [ ]:
EPOCHS_1 = 500
EPOCHS_2 = 500

# initialization/getting reproducible results
reset_seeds()

# Pre-train
initial_learning_rate = 0.020669824682365133
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=math.ceil(num_inputRawTrain/16)*65,
    decay_rate=0.575401007701697,
    staircase=True)
model_pre = tf.keras.Sequential()
checkpoint_filepath = './checkpoint/pre_train/pre_train_weights.hdf5' 
history_pre = train_model(model_pre,inputs_train_raw[0],outputs_train_raw[0],EPOCHS_1)
model_pre.load_weights(checkpoint_filepath) # load the best model
predict_with_FNN(model_pre,inputs_test,outputs_test)
print()

# Train with experimental data
mae_pretrain = []
mae_pretrain_iteration = []
mae_1_pretrain = []
mae_1_pretrain_iteration = []
mae_2_pretrain = []
mae_2_pretrain_iteration = []
pct95th_pretrain = []
pct95th_pretrain_iteration = []
std_pretrain = []
std_pretrain_iteration = []
mse_pretrain = []
mse_pretrain_iteration = []
mse_1_pretrain = []
mse_1_pretrain_iteration = []
mse_2_pretrain = []
mse_2_pretrain_iteration = []
ae_pretrain = []
ae_pretrain_iteration = []

num_data_array = [5,10,30,50,100,150,200,300,400,550]

for x in range(len(num_data_array)):
    # Set learning rate
    initial_learning_rate = 0.020669824682365133
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=math.ceil(num_data_array[x]/16)*65,
        decay_rate=0.575401007701697,
        staircase=True)
    
    mae_pretrain_iteration.append(num_data_array[x])
    mae_1_pretrain_iteration.append(num_data_array[x])
    mae_2_pretrain_iteration.append(num_data_array[x])
    pct95th_pretrain_iteration.append(num_data_array[x])
    std_pretrain_iteration.append(num_data_array[x])
    mse_pretrain_iteration.append(num_data_array[x])
    mse_1_pretrain_iteration.append(num_data_array[x])
    mse_2_pretrain_iteration.append(num_data_array[x])
    ae_pretrain_iteration.append(num_data_array[x])
    for num_train_iteration in range(10):
        # load pre-trained model
        pre_train_model_path = './checkpoint/pre_train/pre_train_weights.hdf5'  
        model_pre.load_weights(pre_train_model_path) # load the best model
        
        # prepare experimental data
        inputs_train_real_cut = inputs_train_real[num_train_iteration][0:0+num_data_array[x]]
        outputs_train_real_cut = outputs_train_real[num_train_iteration][0:0+num_data_array[x]]

        opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

        model_pre.compile(optimizer=opt, loss='mse', metrics=['mse']) 
        checkpoint_filepath = './checkpoint/pre_train/final_weights.hdf5' 
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            verbose = 1,
            monitor='val_mse',
            mode='auto',
            save_best_only=True)
        
        history = model_pre.fit(inputs_train_real_cut, outputs_train_real_cut, epochs=EPOCHS_2, batch_size=16, validation_data=(inputs_test, outputs_test), callbacks=[model_checkpoint_callback])
        model_pre.load_weights(checkpoint_filepath) # load the best model
        predict_results = predict_with_FNN(model_pre,inputs_test,outputs_test)
        print("Number of Data Trained: %.1f" % num_data_array[x])
        print("Iteration Times: %.1f" % num_train_iteration)
        
        mae_pretrain_iteration.append(predict_results[0])
        mae_1_pretrain_iteration.append(predict_results[1])
        mae_2_pretrain_iteration.append(predict_results[2])
        pct95th_pretrain_iteration.append(predict_results[3])
        std_pretrain_iteration.append(predict_results[4])
        mse_pretrain_iteration.append(predict_results[5])
        mse_1_pretrain_iteration.append(predict_results[6])
        mse_2_pretrain_iteration.append(predict_results[7])
        for i in range(num_inputTest):
            ae_pretrain_iteration.extend(predict_results[8][i])
        print()
    mae_pretrain.append(mae_pretrain_iteration)
    mae_pretrain_iteration = []
    mae_1_pretrain.append(mae_1_pretrain_iteration)
    mae_1_pretrain_iteration = []
    mae_2_pretrain.append(mae_2_pretrain_iteration)
    mae_2_pretrain_iteration = []
    pct95th_pretrain.append(pct95th_pretrain_iteration)
    pct95th_pretrain_iteration = []
    std_pretrain.append(std_pretrain_iteration)
    std_pretrain_iteration = []
    mse_pretrain.append(mse_pretrain_iteration)
    mse_pretrain_iteration = []
    mse_1_pretrain.append(mse_1_pretrain_iteration)
    mse_1_pretrain_iteration = []
    mse_2_pretrain.append(mse_2_pretrain_iteration)
    mse_2_pretrain_iteration = []
    ae_pretrain.append(ae_pretrain_iteration)
    ae_pretrain_iteration = []

In [ ]:
# increase the size of the graphs. The default size is (6,4).
plt.rcParams["figure.figsize"] = (9,6)

# list all data in history
print(history.history.keys())

# summarize history for loss
plt.plot(history_pre.history['loss'], '-r.', label='Training Loss')
plt.plot(history_pre.history['val_loss'], '--b.', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.semilogy()
plt.grid()
plt.show()

print(plt.rcParams["figure.figsize"])
#print(history.history)

In [ ]:
# increase the size of the graphs. The default size is (6,4).
plt.rcParams["figure.figsize"] = (9,6)

# list all data in history
print(history.history.keys())

# summarize history for loss
plt.plot(history.history['loss'], '-r.', label='Training Loss')
plt.plot(history.history['val_loss'], '--b.', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.semilogy()
plt.grid()
plt.show()

print(plt.rcParams["figure.figsize"])
#print(history.history)

In [ ]:
# # Creat new directory
# import os
# os.mkdir('results')
import os
if not os.path.exists('results'):
    os.makedirs('results')
# Save Mean Squared Error of pretrained results
with open('./results/mse_results_pretrain.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(mse_pretrain)

# Save Mean Squared Error of non-pretrained results
with open('./results/mse_results_nopre.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(mse_direct)
    
# Save Mean Squared Error of pretrained results
with open('./results/mse_1_results_pretrain.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(mse_1_pretrain)

# Save Mean Squared Error of non-pretrained results
with open('./results/mse_1_results_nopre.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(mse_1_direct)

# Save Mean Squared Error of pretrained results
with open('./results/mse_2_results_pretrain.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(mse_2_pretrain)

# Save Mean Squared Error of non-pretrained results
with open('./results/mse_2_results_nopre.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(mse_2_direct)    
    
# Save Mean Absolute Error of pretrained results
with open('./results/mae_results_pretrain.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(mae_pretrain)

# Save Mean Absolute Error of non-pretrained results
with open('./results/mae_results_nopre.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(mae_direct)
    
# Save Mean Absolute Error of pretrained results
with open('./results/mae_1_results_pretrain.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(mae_1_pretrain)

# Save Mean Absolute Error of non-pretrained results
with open('./results/mae_1_results_nopre.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(mae_1_direct)
    
# Save Mean Absolute Error of pretrained results
with open('./results/mae_2_results_pretrain.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(mae_2_pretrain)

# Save Mean Absolute Error of non-pretrained results
with open('./results/mae_2_results_nopre.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(mae_2_direct)
    
# Save Absolute Error of pretrained results
with open('./results/ae_results_pretrain.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(ae_pretrain)

# Save Absolute Error of non-pretrained results
with open('./results/ae_results_nopre.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(ae_direct)
    
# Save 95th Percentage Error of pretrained results
with open('./results/pct95th_results_pretrain.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(pct95th_pretrain)

# Save 95th Percentage Error of non-pretrained results
with open('./results/pct95th_results_nopre.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(pct95th_direct)
    
# Save Standard Deviation of pretrained results
with open('./results/std_results_pretrain.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(std_pretrain)

# Save Standard Deviation of non-pretrained results
with open('./results/std_results_nopre.csv','w', newline="") as output_file:
    writer = csv.writer(output_file)
    writer.writerows(std_direct)

print("File generation complete!")     